In [ ]:

import numpy as np
from types import SimpleNamespace
from scipy import optimize
par =  SimpleNamespace()
sol = SimpleNamespace()
opt = SimpleNamespace()
# autoreload modules when code is run. Otherwise, python will not see recent changes. 
%load_ext autoreload
%autoreload 2
from inaguralproject import HouseholdSpecializationModelClass

## Question 1 - HF/HM
In the inaugural project we are considering household with a male and a female member. Where we are illustrating how the relative hours worked by the female changes compared to the male due to variations of $\alpha$ and $\sigma$. In Question 1 we are illustrating $\frac{H_F}{H_M}$ for $\alpha \in [0.25,0.5,0.75]$ and $\sigma = [0.5, 1.0, 1.5]$.  

In [ ]:
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D # Used implictely when doing 3D plots
import matplotlib.pyplot as plt # baseline module
from matplotlib import cm # for colormaps
HHS = HouseholdSpecializationModelClass()
N = 3
shape_tuble = (N,N)
alphalist = (0.25,0.50,0.75)
sigmalist = (0.5,1.0,1.5)

alpha_vec = np.empty(shape_tuble)
sigma_vec = np.empty(shape_tuble)
HFHM_vec = np.empty(shape_tuble)


for i, a in enumerate(alphalist):
    for j, s in enumerate(sigmalist):
        HHS.par.alpha = a
        HHS.par.sigma = s
        z = HHS.solve_discrete()
        HFHM = z.HF/z.HM
        alpha_vec[i,j] = a
        sigma_vec[i,j] = s
        HFHM_vec[i,j] = HFHM
     #print(HHS.solve_discrete(do_print=True))
#print(a)
#print(f'LM = {HHS.opt.LM:.3f}')
fig = plt.figure() # create the figure
ax = fig.add_subplot(1,1,1,projection='3d') # create a 3d type axis 
ax.plot_surface(alpha_vec, sigma_vec, HFHM_vec, cmap=cm.jet); # create surface plot in the axis
ax.set_title('Relative Housework')
ax.set_xlabel('$alpha$') # Notice latex syntax
ax.set_ylabel('$sigma$')
ax.set_zlabel(r'$\frac{H_{F}}{H_{M}}$')
# note: fig.add_subplot(a,b,c) creates the c'th subplot in a grid of a times b plots

# Interpretation of Q1
The parameter $\alpha$ represents the share of time spent by the male on the market working. The female's time inpus is raised by $1-\alpha$. The illustration is a 3D plot that shows the relation between $\frac{H_F}{H_M}, \alpha$ and $\sigma$. The surface creates the relationship between the variables. $\frac{HF}{HM}$ is the ratio of time spent on hours working at home, where $>1$ indicates that the female member spends more time on house work than the male.  

## Question 2 - Discrete time
Illustration of $\log{\frac{H_M}{H_F}}$ against $\log{\frac{w_F}{w_M}}$ for $w_F \in [0.8, 0.9, 1, 1.1, 1.2]$, where $w_F$ represents the different wages.

In [ ]:
# a. reset the parameter values
HHS.par.sigma = 1
HHS.par.alpha = 0.5 
wflist = (0.8, 0.9, 1.0, 1.1,1.2)

# b. create the figure
fig = plt.figure()

# c. create the plot 
ax = fig.add_subplot(1,1,1)

# d. create empty vectors for the independent and dependent values
lnwFwM_vec = np.empty(5)
lnHFHM_vec = np.empty(5)

# e. fill the vectors
for i, wf1 in enumerate(wflist):
    HHS.par.wF = wf1
    lnwFwM = np.log(HHS.par.wF/HHS.par.wM)
    x = HHS.solve_discrete()
    lnHFHM = np.log(x.HF/x.HM)
    lnwFwM_vec[i] = lnwFwM
    lnHFHM_vec[i] = lnHFHM

# f. plot
ax.plot(lnwFwM_vec,lnHFHM_vec) # c is the color, the argument is the color in rgb format (in percentages of 255)    
ax.scatter(lnwFwM_vec,lnHFHM_vec)
ax.set_title('Relative Housework against Relative Wages')
ax.set_xlabel(r'$log\left(\frac{w_{F}}{w_{M}}\right)$')
ax.set_ylabel(r'$log\left(\frac{H_{F}}{H_{M}}\right)$')

# Interpretation of Q2
The illustration from Question 2 shows the relation between the relative wages of woman and man and the relative amount of housework they do. The x-axis shows the log ratio of the womans and the man's work at the wage. The y-axis represents the log ratio of the woman and the man's work at the house. The scatter shows the data points given and the line shows the trend between the data points. We can see a negative relationship between the relative wage and the relative housework. In other words, it suggests that the woman's wages increase relative to the man's wages, their relative share of housework decreases.

## Question 3 - Continuous time
It is now assumes that choice made are continuous and given as $L_M, L_F, H_M, H_F \in [0,24].

In [ ]:
# a. reset the parameter values
HHS.par.sigma = 1
HHS.par.alpha = 0.5 
wflist = (0.8, 0.9, 1.0, 1.1,1.2)

# b. create the figure
fig = plt.figure()

# c. create the plot 
ax = fig.add_subplot(1,1,1)

# d. create empty vectors for the independent and dependent values
lnwFwM_vec = np.empty(5)
lnHFHM_vec = np.empty(5)

# e. fill the vectors
for i, wf1 in enumerate(wflist):
    HHS.par.wF = wf1
    lnwFwM = np.log(HHS.par.wF/HHS.par.wM)
    y = HHS.solve_continue()
    lnHFHM = np.log(y.HF/y.HM)
    print(lnHFHM)
    lnwFwM_vec[i] = lnwFwM
    lnHFHM_vec[i] = lnHFHM


# f. plot
ax.plot(lnwFwM_vec,lnHFHM_vec) # c is the color, the argument is the color in rgb format (in percentages of 255)    
ax.scatter(lnwFwM_vec,lnHFHM_vec)
ax.set_title('Relative Housework against Relative Wages')
ax.set_xlabel(r'$log\left(\frac{w_{F}}{w_{M}}\right)$')
ax.set_ylabel(r'$log\left(\frac{H_{F}}{H_{M}}\right)$')

# Interpretation of Q3
The plot shows the relationship between the log of relative wages ($\log(w_F/w_M)$) and the log of the magnitude of relative housework ($\log(H_F/H_M)$) in a household where the female and male have different levels of productivity and face different wage rates. There are 5 points, which is the results from the female wages. As the relative wage ratio increases, the magnitude of relative housework also increases. The slope of the line is negative, indicating a negative relationship between the two variables.

## Question 4
Siminski and Yetsenga (2022) presents the following regression in run on time use data couples:

$\log{\frac{H_F}{H_M}} = \beta_0 +\beta_1 \log{\frac{w_F}{w_M}}$

and find the best results to be $\beta_0 \approx 0.4$ and $\beta_1 \approx -0.1$. 
We are creating a for loop to run through all the combinations of $\alpha$ and $\sigma$.

In [ ]:
best_res = float('inf')

best_alpha = None
best_sigma = None
best_beta0 = None
best_beta1 = None

alpha_list = np.linspace(0.1,1,10)
sigma_list = np.linspace(0.1,2,20)

# a. run Question 3 through a for-loop with varying alpha and sigma values
for a in alpha_list:
    for s in sigma_list:
        HHS = HouseholdSpecializationModelClass()
        HHS.par.sigma = a
        HHS.par.alpha = s
        wflist = (0.8, 0.9, 1.0, 1.1, 1.2)

        # b. create empty vectors for the independent and dependent value
        lnwFwM_vec = np.empty(5)
        lnHFHM_vec = np.empty(5)

        # c. fill the vector
        for i, wf1 in enumerate(wflist):
            HHS.par.wF = wf1
            lnwFwM = np.log(HHS.par.wF / HHS.par.wM)
            y = HHS.solve_continue()
            lnHFHM = np.log(y.HF / y.HM)
            lnwFwM_vec[i] = lnwFwM
            lnHFHM_vec[i] = lnHFHM
        # d. run the regression
        x = lnwFwM_vec
        y = lnHFHM_vec
        A = np.vstack([np.ones(x.size), x]).T
        HHS.sol.beta0, HHS.sol.beta1 = np.linalg.lstsq(A, y, rcond=None)[0]
        new_res = (0.4 - HHS.sol.beta0) ** 2 + (-0.1 - HHS.sol.beta1) ** 2
        # e. check if the sum of the squared errors are smaller than the previous best estimate
        if new_res < best_res:
            best_res = new_res
            best_alpha = a
            best_sigma = s
            best_beta0 = HHS.sol.beta0
            best_beta1 = HHS.sol.beta1

# f. print results
print(f'The minimizing parameter values are: alpha = {best_alpha:.4f}, sigma = {best_sigma:.4f}\nwhich results in the beta values: beta0 = {best_beta0:.4f}, beta1 = {best_beta1:.4f}\nThe sum of the squared errors are therefore: {best_res:.4f}')


# g. fill vectors to present the model values of our model and Siminski and Yetsenga (2022) 
our_values = np.empty(5)
Sim_Yet_values = np.empty(5)
for i, wf1 in enumerate(wflist):
    HHS.par.wF = wf1
    lnwFwM = np.log(HHS.par.wF / HHS.par.wM)
    our_values[i] = best_beta0+best_beta1*np.log(wf1)
    Sim_Yet_values[i] = 0.4-0.1*np.log(wf1)



# h. create the figure
fig = plt.figure()

# i. create the plot 
ax = fig.add_subplot(1,1,1)

ax.plot(lnwFwM_vec,our_values,label='Our',c=(0.,0.1,0.1)) # c is the color, the argument is the color in rgb format (in percentages of 255)    
ax.plot(lnwFwM_vec,Sim_Yet_values, label='Sim_Yet',c=(0.,0.1,0.2)) # c is the color, the argument is the color in rgb format (in percentages of 255)    
ax.set_title('Relative Housework against Relative Wages')
ax.set_xlabel(r'$log\left(\frac{w_{F}}{w_{M}}\right)$')
ax.set_ylabel(r'$log\left(\frac{H_{F}}{H_{M}}\right)$')
ax.legend(loc='upper right',prop={'size':9}); # Activate legend (uses the labels in ax.plot())

## Interpretation of Q4
We got the results for the minimizing parameters of $\alpha = 0.3$ and $\sigma = 1.7$ and with some approximation of the beta-values of $\beta_0 = 0.383$ and $\beta_1 = -0.0$ and the sum of squared errors to be 0.0104.